In [ ]:
!pip install \
    langchain_community\
    langchain_text_splitters\
    langchain_openai\
    langchain_pinecone


In [ ]:
!pip install pypdf

In [ ]:
import os
import numpy as np
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings

In [ ]:
from urllib.request import urlretrieve

In [ ]:
files = [
    "https://www.deeplearningbook.org/front_matter.pdf",
    "https://www.microsoft.com/en-us/research/uploads/prod/2006/01/Bishop-Pattern-Recognition-and-Machine-Learning-2006.pdf",
    "https://www.academia.dk/BiologiskAntropologi/Epidemiologi/DataMining/Artificial_Intelligence-A_Guide_to_Intelligent_Systems.pdf",
    "https://mrce.in/ebooks/AI%20Foundations%20of%20Computational%20Agents%203rd%20Ed.pdf"
]
os.makedirs("data",exist_ok=True)

In [ ]:
for url in files:
    file_path=os.path.join("data", url.rpartition("/")[2])
    urlretrieve(url,file_path)

In [ ]:
loader = PyPDFDirectoryLoader("./data/")

In [ ]:
docs_before_split=loader.load()

In [ ]:
docs_before_split[0]

In [ ]:
from google.colab import userdata
os.environ['OPENAI_API_KEY']=userdata.get('OPENAI_API_KEY')
os.environ['PINECONE_API_KEY']=userdata.get('PINECONE_API_KEY')

In [ ]:
embeddings =OpenAIEmbeddings(
    model="text-embedding-3-small",
)

index_name="chatbot"

In [ ]:
text_splitter=RecursiveCharacterTextSplitter()
split_docs=text_splitter.split_documents(docs_before_split)

In [ ]:
split_docs[25]

In [ ]:
vectorestore=PineconeVectorStore.from_documents(split_docs,embeddings,index_name=index_name)

In [ ]:
query = "What is the neural network?"

In [ ]:
similar_docs=vectorestore.similarity_search(query)

In [ ]:
similar_docs

In [ ]:
llm= ChatOpenAI(
    model="gpt-4o",
    temperature=0.3
)

In [ ]:
qa= RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorestore.as_retriever()
)

In [ ]:
qa.invoke(query)

In [ ]:
!pip install gradio

In [ ]:
!pip install langgraph

In [ ]:
import gradio as gr
from langchain_core.runnables.history import RunnableWithMessageHistory


def chat_with_bot(user_message, history):
    conversation_chain = RunnableWithMessageHistory(llm=llm, history=history)
    if not user_message.strip():
        return gr.update(value="Message cannot be empty!"), history


    response = conversation_chain.predict(input=user_message)

    history.append((user_message, response))

    return gr.update(value=""), history

with gr.Blocks() as app:
    gr.Markdown("# AI Learning Chatbot")

    chatbot = gr.Chatbot(label="Chat Interface")
    user_input = gr.Textbox(label="Your Message", placeholder="Type your question here...", lines=1)
    clear_button = gr.Button("Clear Chat")
    chat_history = []

    def user(user_message, history):
        if not user_message.strip():
            return gr.update(value="Message cannot be empty!"), history

    user_input.submit(chat_with_bot, [user_input, chatbot], [user_input, chatbot], queue=True)
    clear_button.click(lambda: [], None, chatbot, queue=False) # changed lambda to return []


if __name__ == "__main__":
    app.launch(debug=True)



/usr/local/lib/python3.11/dist-packages/gradio/components/chatbot.py:273: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://c4c6bf8b90fc4ca011.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
